<img src="data/ccdproc_banner.svg" width=500 alt="ccdproc logo">

# ccdproc

+ Code: https://github.com/astropy/ccdproc
+ Documentation: https://ccdproc.readthedocs.io/
+ Detailed image reduction guide using ccdproc: https://github.com/astropy/ccd-reduction-and-photometry-guide
+ Report bugs/issues here: https://github.com/astropy/ccdproc/issues
    - Most of the bugs reported at or after the January AAS workshop are fixed in ccdproc 2.2 😃

## In this section we will

+ learn two interfaces for combining images
+ talk through an overview of the combination methods
+ preview of changes coming to address current performance bottlenecks

<hr>

## Preliminaries

In [ ]:
# If you work with files regularly then you should use 
# pathlib, which is part of the Python standard library
from pathlib import Path

import numpy as np

# Initial imports -- note the import of CCDData from astropy 
from astropy.nddata import CCDData
from astropy import units as u
from astropy.stats import mad_std

import ccdproc as ccdp

from matplotlib import pyplot as plt

# This displays static plots in the notebook...it will always work
%matplotlib inline

# Use this instead for interactive plots, but note that you may need 
# to install ipympl for it to work

# %matplotlib widget

# This function display reasonably nicely with minimal effort
from convenience_functions import show_image

#### Generate some images to use in this tutorial 

To minimize the downloads necessary the next cell generates 10 copies of the single bias image included in this notebook, adding some random noise along the way.

In [ ]:
# You do NOT need to understand this cell to do the rest of the
# tutorial (though it is another example of using ccdproc!)

# Make a folder for the images
combination_path = Path('images-combine')
combination_path.mkdir(exist_ok=True)

# Load the bias image that comes with the tutorial
bias = CCDData.read('data/combined_bias.fit.gz')

n_for_combining = 10

# this is roughly correct for the camera I use most often
read_noise = 7

for i in range(n_for_combining):
    # Copy the original bias
    image_with_noise = bias.copy()  
    
    # Generate some noise the same shape as the data...
    noise = read_noise * np.random.standard_normal(bias.shape)
    
    # ...and add it to the data
    image_with_noise.data = image_with_noise.data + noise
    
    # Save the image
    image_with_noise.write(combination_path / 'bias-{}.fit'.format(i))

## Interface to image combination

`ccdproc` provides two different interfaces for combining images: a `combine` function and a `Combiner` class. 

+ In most cases, the `combine` function (detailed description [at this link](https://ccdproc.readthedocs.io/en/latest/api/ccdproc.combine.html#ccdproc.combine)), should be used. It allows users to limit the amount of memory used by processing images in chunks and provides access to all of the features of `ccdproc`. The documentation on the function is limited, however.

+ The `Combiner` class (description [here](https://ccdproc.readthedocs.io/en/latest/image_combination.html) and in more detail [here](https://ccdproc.readthedocs.io/en/latest/api/ccdproc.Combiner.html#ccdproc.Combiner)) is where all of the functionality is actually implemented. 

This tutorial will primarily demonstrate how to use the `Combiner` class because that it is far easier to break that into pieces, and because understanding those pieces makes it easier to understand what the `combine` function does.

## Approaches to image combination

There are a few elements of image combination that, together, can make it complex and time consuming:

+ Excluding extreme pixels (image clipping): Drop the min/max pixels in the stack, drop pixels above or below a particular value, sigma clip.
+ Operate on the images before combination: Science images may need to be aligned before combining; calibration images typically do not.
+ Combination method: Average the images, or sum them, or find the median.

This tutorial will not get into which choices you should make. Some general advice is in the [CCD data reduction guide section on image combination](https://mwcraig.github.io/ccd-as-book/01-06-Image-combination.html), and examples of image combination in of each of the calibration steps are in the guide also (for example, in the [section on comibing bias images](https://mwcraig.github.io/ccd-as-book/02-04-Combine-bias-images-to-make-master.html)).

When using `ccdproc` you need to exclude extreme pixels and perform any operations like reprojecting before combining, though the `combine` function takes care of this detail for you.

### Creating a `Combiner`

The first step in combining images using the object interface is to create a `Combiner` from a list of `CCDData` images.

That is done here using an `ImageFileCollection` to read the data.

In [ ]:
ifc = ccdp.ImageFileCollection(combination_path)

images_to_combine = [ccd for ccd in ifc.ccds()]

combiner = ccdp.Combiner(images_to_combine)

### Removing pixels with extreme values

A few methods are provided for excluding extreme values:

+ min/max clipping removes the pixels above and/or below values set by the user.
+ extrema clipping removes the largest and/or smalest value(s) from the image stack before clipping.
+ sigma clipping removes pixels that are further than a user-set number of deviations from a central value.

Each of these is described in more detail in the [ccdproc documentation](https://ccdproc.readthedocs.io/en/latest/image_combination.html#image-masks-and-clipping).

The example below shows how to do implement a common choice: sigma clipping those values more than a few median absolute deviations (set by the `dev_func` argument below) from the median value (set by the `func` argument below).

In [ ]:
combiner.sigma_clipping(low_thresh=5, high_thresh=5, 
                        func=np.ma.median, dev_func=mad_std)

Though there is not any visible output here, the `combiner` has internally marked which pixels should be excluded from the combination.

Several combination methods can be combined. For example, to also exclude the highest and lowest values of the pixels not masked by sigma clipping:

In [ ]:
combiner.clip_extrema(nlow=1, nhigh=1)

### Combining the images

Three methods are provided for combining images:

+ Average the images, taking into account the number of images actually included in the combination. Weights can be provided.
+ Add the images in the stack
+ Calculate the median value of the images stack

The methods are similar to each other; averaging is shown below.

In [ ]:
combined_images = combiner.average_combine()

### Before and after

The image on the left below is a single bias image and the image on the right is the combination.

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True, tight_layout=True, figsize=(20, 10))

# Dsiplay the first of the bias images
show_image(images_to_combine[0].data, ax=axes[0], fig=fig)
axes[0].set_title('Single bias images')

# Display the combined image -- the perceintile for the low end of the color bar is set
# so as to make the range of the data the same in each case.
show_image(combined_images, ax=axes[1], fig=fig, percl=99.5)
axes[1].set_title('Combined biases')

Note that:

+ The noise is reduced in the combined images.
+ There is a faint vertical line around `x = 135` that is swamped by noise in the individual images but shows up more clearly in the combined image.

### Summary of other features

+ Images can be scaled to the same central (average or median) value before combining. This is useful for creating twilight or sky flats. See this example from the [ccdproc documentation](https://ccdproc.readthedocs.io/en/latest/image_combination.html#combination-with-image-scaling) or this [discussion in the CCD guide](https://mwcraig.github.io/ccd-as-book/05-04-Combining-flats.html).

+ Though `ccdproc` itself cannot currently shift, align, or reproject science images before combining them, an example showing how to do so is [in the documentation](https://ccdproc.readthedocs.io/en/latest/image_combination.html#combination-with-image-transformation-and-alignment).


### Limitations and planned improvements

Some of the current limitations and planned improvements are:

+ The memory required can be as large as a few times the size of the images being combined when using the `Combiner` interface. Use the `combine` function instead to control memory use.
    + Memory use will be better controlled in many cases in the next release.
+ Using average or sum to combine images is unnecessarily slow.
    + This will be improved by a factor of a few in the next release.
+ Use of medians in clipping or combination is slow. 
    + This is more challenging, but efforts are underway to write a small package with optimized code for performing these operations on astronomical images. 
    + When that is ready, it will be incorporated into `ccdproc`